# Geographic distributions in OCRE

This notebook looks at the geography of minting in two different ways:

- distribution of issues by mint throughout OCRE
- distribution of issues by mint by emperor or issuing authority.  

It uses version `3.1.1` of the `nomisma` library.



## Configure Jupyter notebook

First configure the Jupyter notebook. In addition to the `nomisma` library, we will use `plotly` for graph plots, and a `histoutils` package to simplify working with histograms.

In [11]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [12]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::nomisma:3.1.1`
import $ivy.`edu.holycross.shot::histoutils:2.2.0`
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                                      

## Load the full OCRE data set

In [13]:
import edu.holycross.shot.nomisma._
val ocreCex = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"
val ocre = OcreSource.fromUrl(ocreCex)

// Sanity check:
println(ocre.size + " records loaded.") 


2019-12-27 23:31:50.782Z  info [OcreSource] Reading 50644 lines of CEX data.  - (OcreSource.scala:21)
2019-12-27 23:31:51.616Z  info [OcreSource] Created Ocre with 50644 issues.  - (OcreSource.scala:33)


50644 records loaded.


import edu.holycross.shot.nomisma._

ocreCex: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"
ocre: Ocre = Ocre(
  Vector(
...

Set up environment for graph plotting with ploty.

## Load lon-lat data for mints 



In [15]:
import scala.io.Source
val mintsCsv = "https://raw.githubusercontent.com/neelsmith/nomisma/master/tables/mintpoints.csv"
val mintsData = Source.fromURL(mintsCsv).getLines.toVector


import scala.io.Source

mintsCsv: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/tables/mintpoints.csv"
mintsData: Vector[String] = Vector(
  "mint,lon,lat",
...

In [16]:
val mints = MintPointCollection(mintsData.mkString("\n"), ",")
println("Number mints: " + mints.size)

val ocreGeo = Ocre(ocre.issues, mints)

Number mints: 1813


mints: MintPointCollection = MintPointCollection(
  Vector(
...
ocreGeo: Ocre = Ocre(
  Vector(
...

In [17]:
val countsForMints = ocreGeo.hasMint.issues.groupBy(_.mint).map{ case (mint, iss) => (mint, iss.size)}
                                                                                                                           
//

countsForMints: Map[String, Int] = Map(
  "ravenna_com" -> 8,
...

In [18]:
val geoCsv = for (mnt <- countsForMints.keySet.toVector) yield {
    val geoData = mints.forMint(mnt)
    geoData match {
        case None => {
            println("No geo data for " + mnt)
            ""
        }
        case _ =>  {
            val csv = mnt + "," + geoData.get.pt + "," + countsForMints(mnt)
            //println(csv)
            csv
        }
    }  
}
val validGeo = geoCsv.filter(_.nonEmpty)
println(validGeo.size + " locatable mints")
println("mint,lon,lat,issues")
println(validGeo.mkString("\n"))

//mints.forMint("rome").get.map(pt => pt.mint + "," + pt.pt)
//println(mints.mintPoints.filter(_.mint == "ravenna"))

No geo data for southern_gallic_mint
No geo data for comitatensian_mint
No geo data for gallia
No geo data for eastern_mint_ric
No geo data for peloponnesus
No geo data for nicaea_gallia
No geo data for asia
49 locatable mints
mint,lon,lat,issues
ravenna_com,12.196604,44.415718,8
ravenna,12.196604,44.415718,184
lugdunum,4.819444,45.759722,2862
carthage,10.312944,36.847009,155
pergamum,27.183333,39.116667,42
carnuntum,16.866667,48.116667,11
caesareia_cappadocia,35.483333,38.733333,35
barcino,2.169911,41.387911,4
ticinum,9.156562,45.185899,946
tripolis_phoenicia,35.8387756,34.4290008,24
treveri,6.641389,49.756667,2647
arelate,4.630799,43.677616,1070
siscia,16.371388,45.483168,2449
antiocheia_syria,36.15,36.2,2387
colonia_claudia_ara_agrippinensium,6.966667,50.95,144
sirmium,19.610106,44.966447,248
commagene,36.74,37.25,2
caesaraugusta,-0.876,41.657,54
heraclea_thracica,27.95528,40.97003,564
alexandreia_egypt,29.909773,31.201435,591
camulodunum,0.901298,51.889852,360
asia_minor,29.53125,3

geoCsv: Vector[String] = Vector(
  "ravenna_com,12.196604,44.415718,8",
...
validGeo: Vector[String] = Vector(
  "ravenna_com,12.196604,44.415718,8",
...

## Visualize issues per mint per authority

Start with a chronologically ordered list of `Ocre` objects grouped by issuing authority:

In [19]:
val byAuthorityChronological = ocre.hasMint.byAuthority
val authNames = byAuthorityChronological.map(_._1)
val authOcres =  byAuthorityChronological.map(_._2)


byAuthorityChronological: Vector[(String, Ocre)] = Vector(
  (
...
authNames: Vector[String] = Vector(
  "augustus",
...
authOcres: Vector[Ocre] = Vector(
  Ocre(
...

In [34]:
val summaries = for (auth <- authNames) yield {

    val ocreForAuth =  Ocre(ocre.issuesForAuthority(auth))
    val countsForMints = ocreForAuth.hasMint.issues.groupBy(_.mint).map{ case (mint, iss) => (mint, iss.size)}
    val csvLines = for (mintCount <- countsForMints) yield {
        val mnt =  mintCount._1
        val issueCount =  mintCount._2
        val geoData = mints.forMint(mnt)
        geoData match {
            case None => {
                println("No geo data for " + mnt)
                ""
            }
            case _ =>  {
                val csv = auth + "," + mnt + "," + geoData.get.pt + "," + issueCount
                //println(csv)
                csv
            }
        }   
    }
    csvLines.toVector.filter(_.nonEmpty)
}



No geo data for peloponnesus
No geo data for gallia
No geo data for eastern_mint_ric
No geo data for asia
No geo data for asia
No geo data for gallia
No geo data for southern_gallic_mint
No geo data for southern_gallic_mint
No geo data for gallia
No geo data for nicaea_gallia
No geo data for comitatensian_mint


summaries: Vector[Vector[String]] = Vector(
  Vector(
...

In [36]:
println("authority,mint,lon,lat,issues")
println(summaries.flatten.mkString("\n"))



authority,mint,lon,lat,issues
augustus,lugdunum,4.819444,45.759722,111
augustus,pergamum,27.183333,39.116667,40
augustus,treveri,6.641389,49.756667,1
augustus,caesaraugusta,-0.876,41.657,54
augustus,nemausus,4.360278,43.834875,8
augustus,ephesus,27.340833,37.939722,11
augustus,emerita,-6.34567,38.916159,15
augustus,samos,26.943056,37.689444,1
augustus,antiocheia_pisidia,31.189704,38.304239,3
augustus,colonia_patricia,-4.779171,37.884683,131
augustus,rome,12.5,41.9,195
augustus,cyrene,21.85727,32.823041,5
tiberius,caesareia_cappadocia,35.483333,38.733333,5
tiberius,commagene,36.74,37.25,2
tiberius,rome,12.5,41.9,51
tiberius,lugdunum,4.819444,45.759722,32
gaius,caesareia_cappadocia,35.483333,38.733333,5
gaius,rome,12.5,41.9,46
gaius,lugdunum,4.819444,45.759722,12
claudius,lugdunum,4.819444,45.759722,1
claudius,pergamum,27.183333,39.116667,2
claudius,caesareia_cappadocia,35.483333,38.733333,5
claudius,ephesus,27.340833,37.939722,3
claudius,rome,12.5,41.9,115
nero,caesareia_cappadocia,35.4

In [35]:
// If you're running this locally and want to save the CSV data to a file,
// uncomment these two lines:
//import java.io.PrintWriter
//new PrintWriter("mint-issues-by-auth.csv"){write("authority,mint,lon,lat,issues\n" + summaries.flatten.mkString("\n")); close;}



import java.io.PrintWriter

res34_1: PrintWriter = ammonite.$sess.cmd34$Helper$$anon$1@46962f1f